In [1]:
import pyodbc
import pandas as pd
import numpy as np
import spacy
import spacy_fastlang


In [2]:
server = 'frbineprdsqlpool01.database.windows.net'
database = 'frdw1'
driver = '{ODBC Driver 17 for SQL Server}'

conn_str = f'''
    DRIVER={driver};
    SERVER={server};
    DATABASE={database};
    Authentication=ActiveDirectoryInteractive;
'''

try:
    conn = pyodbc.connect(conn_str)
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")


Error: <class 'pyodbc.Error'> returned a result with an exception set


In [ ]:
sql_query = """SELECT
BOOK.PNR
--,BOOK.BookingSK
--,BOOK.OutboundDepartureDate
--,BOOK.InboundDepartureDate
--,RESP.QuestionID
,BOOK.JourneyType
,BOOK.OutboundDepartureAirport
,BOOK.OutboundArrivalAirport
,RESP.ResponseID
,Heading
,Entered_AnswerText
FROM ctrack_rpt.sm_responses RESP
LEFT JOIN [ctrack_rpt].[sm_surveyquestions] QUEST ON QUEST.QuestionID = RESP.QuestionID
LEFT JOIN [ctrack_rpt].[sm_responses_custvar] CUSTVAR ON CUSTVAR.ResponseID = RESP.ResponseID
LEFT JOIN [revenue_rpt].[DimBooking] DBOOK ON DBOOK.BookingNK = CUSTVAR.BookingSK
LEFT JOIN [revenue_rpt].[vBookingsModelled] BOOK ON DBOOK.BookingSK = BOOK.BookingSK

WHERE RESP.SurveyID = 517769945
AND Entered_AnswerText != ''

--RESP.QuestionID IN (
--184779188,
--184779189,
--184779213
--)
AND Response_CreatedDate BETWEEN '2024-07-01' AND '2024-08-31'
AND BOOK.BookingCultureDescription LIKE ('English%%')

ORDER BY RESP.ResponseID ASC"""

In [ ]:
df = pd.read_sql(sql_query, conn)
df

In [ ]:
df.to_csv(r'C:\Users\quercios\OneDrive - Ryanair Ltd\Project Data Analyst\CSAT NLP\Python\res\open_ended_df.csv')

In [ ]:
df = pd.read_csv(r'C:\Users\quercios\OneDrive - Ryanair Ltd\Project Data Analyst\CSAT NLP\Python\res\open_ended_df.csv')

In [ ]:
df

In [ ]:
# Load SpaCy model and add the language detector pipeline
nlp = spacy.load("en_core_web_lg")
nlp.add_pipe("language_detector")  # Add the language detector to the pipeline

In [ ]:
# Define a function that processes the text and returns the language
def detect_language_and_score(text):
    doc = nlp(str(text))  # Ensure text is a string
    detected_language = doc._.language
    #language_score = doc._.language_score
    return detected_language

In [ ]:
detected_languages = []

for i,t in enumerate(df["Entered_AnswerText"]):
    detected = detect_language_and_score(t)
    detected_languages.append(detected)



In [ ]:
df["Detected Language"] = detected_languages

In [ ]:
df

In [ ]:
df.to_csv(r'C:\Users\quercios\OneDrive - Ryanair Ltd\Project Data Analyst\CSAT NLP\Python\res\open_ended_df_detected.csv')

In [2]:
df = pd.read_csv(r'C:\Users\quercios\OneDrive - Ryanair Ltd\Project Data Analyst\CSAT NLP\Python\res\open_ended_df_detected.csv')
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,PNR,JourneyType,OutboundDepartureAirport,OutboundArrivalAirport,ResponseID,Heading,Entered_AnswerText,Detected Language
0,0,0,0,ESTF3B,One Way,EDI,LPA,118637791724,We are happy to hear you were satisfied with y...,Nice smooth flight with friendly staff,en
1,1,1,1,UFW4SY,One Way,KRK,MAN,118637815104,We are happy to hear you were satisfied with y...,"On time, friendly, efficient.",fr
2,2,2,2,GSGG4E,One Way,MAN,MXP,118637821608,Would you like to share anything else about yo...,No,en
3,3,3,3,GSGG4E,One Way,MAN,MXP,118637821608,We are happy to hear you were satisfied with y...,Maria,en
4,4,4,4,EP3MFN,One Way,CTA,ATH,118637842181,We are happy to hear you were satisfied with y...,It met my expectation. I did not expect grandi...,en
...,...,...,...,...,...,...,...,...,...,...,...
457486,457486,457486,457486,YJ98JL,One Way,FCO,AGP,118681752174,We are happy to hear you were satisfied with y...,"Unexpected happens, prices are really good",en
457487,457487,457487,457487,YJ98JL,One Way,FCO,AGP,118681752174,Would you like to share anything else about yo...,Not happy that luggage has 20k limit,en
457488,457488,457488,457488,DPY6HN,One Way,PIS,STN,118681789467,We are sorry to hear you were not satisfied wi...,The only let down was the flight was delayed b...,en
457489,457489,457489,457489,ON8PJZ,One Way,ZAD,BER,118684058198,Would you like to share anything else about yo...,stop with the fees for check in at the ariport...,en


In [4]:
df = df.drop('Unnamed: 0.1', axis =1)
df = df.drop('Unnamed: 0.2', axis =1)
df = df.drop('Unnamed: 0', axis =1)

KeyError: "['Unnamed: 0.1'] not found in axis"

In [ ]:
df

In [6]:
df_en = df[df['Detected Language']== 'en'].copy()
df_en

,PNR,JourneyType,OutboundDepartureAirport,OutboundArrivalAirport,ResponseID,Heading,Entered_AnswerText,Detected Language
0,ESTF3B,One Way,EDI,LPA,118637791724,We are happy to hear you were satisfied with y...,Nice smooth flight with friendly staff,en
2,GSGG4E,One Way,MAN,MXP,118637821608,Would you like to share anything else about yo...,No,en
3,GSGG4E,One Way,MAN,MXP,118637821608,We are happy to hear you were satisfied with y...,Maria,en
4,EP3MFN,One Way,CTA,ATH,118637842181,We are happy to hear you were satisfied with y...,It met my expectation. I did not expect grandi...,en
5,PQ4C9K,Round Trip,RHO,BGY,118637862714,We are happy to hear you were satisfied with y...,more than satisfied from the whole * servicing...,en
...,...,...,...,...,...,...,...,...
457486,YJ98JL,One Way,FCO,AGP,118681752174,We are happy to hear you were satisfied with y...,"Unexpected happens, prices are really good",en
457487,YJ98JL,One Way,FCO,AGP,118681752174,Would you like to share anything else about yo...,Not happy that luggage has 20k limit,en
457488,DPY6HN,One Way,PIS,STN,118681789467,We are sorry to hear you were not satisfied wi...,The only let down was the flight was delayed b...,en
457489,ON8PJZ,One Way,ZAD,BER,118684058198,Would you like to share anything else about yo...,stop with the fees for check in at the ariport...,en


In [7]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob    

In [8]:
# Load SpaCy model and add the language detector pipeline
nlp = spacy.load("en_core_web_lg")
nlp.add_pipe('spacytextblob')

c:\Users\quercios\OneDrive - Ryanair Ltd\Desktop\Docs\Python\myenv\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_lg' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\Users\quercios\OneDrive - Ryanair Ltd\Desktop\Docs\Python\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def detect_polarity(text):
    doc = nlp(str(text))  # Ensure text is a string
    detected_polarity = doc._.blob.polarity
    #language_score = doc._.language_score
    return detected_polarity


detected_polarity = []

for i,t in enumerate(df_en["Entered_AnswerText"]):
    print(i)
    detected = detect_polarity(t)
    detected_polarity.append(detected)

In [ ]:
df_en["Detected Polarity"] = detected_polarity

In [ ]:
df_en

In [ ]:
df_en.to_csv(r'C:\Users\quercios\OneDrive - Ryanair Ltd\Project Data Analyst\CSAT NLP\Python\res\open_ended_df_detected_polarity.csv')

In [9]:
df_en = pd.read_csv(r'C:\Users\quercios\OneDrive - Ryanair Ltd\Project Data Analyst\CSAT NLP\Python\res\open_ended_df_detected_polarity.csv')

In [10]:
df_en.sort_values("Detected Polarity")

,Unnamed: 0,PNR,JourneyType,OutboundDepartureAirport,OutboundArrivalAirport,ResponseID,Heading,Entered_AnswerText,Detected Language,Detected Polarity
18980,20118,CPGMHX,Round Trip,BHX,ALC,118646672754,We are sorry you were not happy with the frien...,Staff really miserable on return boarding gate,en,-1.0
38209,40461,ZPDUKG,One Way,BHX,DUB,118647803896,We are sorry to hear you were not satisfied wi...,The flight were delayed at 2 times..it makes m...,en,-1.0
83400,88220,FICQYV,Round Trip,STN,JTR,118652026581,Would you like to share anything else about yo...,Do not board people and make them wait on the ...,en,-1.0
315993,335076,VP8GYD,One Way,RMI,STN,118672099138,Would you like to share anything else about yo...,Your staff at Stanstead are utterly miserable,en,-1.0
83356,88172,VH24XB,Round Trip,STN,BUD,118652026177,We are happy to hear you were satisfied with y...,Flight crew were terrible,en,-1.0
...,...,...,...,...,...,...,...,...,...,...
64849,68607,KN2S2J,One Way,BFS,STN,118650113573,We are happy to hear you were satisfied with y...,It’s best flight in a world,en,1.0
64847,68605,CTNPJR,One Way,ALC,MAN,118650113333,We are happy to hear you were satisfied with y...,Excellent service,en,1.0
170372,180687,NR3U2K,One Way,PMI,OTP,118659083106,Would you like to share anything else about yo...,Everything perfect,en,1.0
357266,378870,EGME8L,Round Trip,BFS,AGP,118675277219,Would you like to share anything else about yo...,"Yes,taking off at the stated time would be the...",en,1.0
